In [1]:
import os
import time
import json

import concurrent
import numpy as np
import pandas as pd
from tqdm import tqdm
from openai import OpenAI, AzureOpenAI
from sentence_transformers import SentenceTransformer

In [2]:
config = json.load(open('./config.json'))

organization = ""

api_source = 'OpenAI'

api_key = config[api_source]['openai_api_key'] #constants.AZURE_OPENAI_KEY
api_endpoint = config[api_source]['openai_api_endpoint'] 

gpt_model = config[api_source]["gpt_model"]
temperature = 0

In [76]:
nruns = 3

system_options = {
    "medical-expert-examples": """
    You are a medical professional. You will be asked to relate two conditions through 'is a' statements.
    For example, Haematemesis is GI bleeding.
    For example, Acute myocardial infarction is Acute myocardial infarction.
    For example, Hepatitis is Acute Liver Injury.
    You will be given similar statements and asked to respond with only "True" or "False".
    """,
}

prompt_options = {
    "is-a": """
{} is {}.
"""
}

In [77]:
system_name = "medical-expert-examples"
system_content = system_options[system_name]

prompt_name = "is-a"
prompt = prompt_options[prompt_name]

In [8]:
adr_section = pd.read_csv('/Users/undinagisladottir/Documents/Columbia/Tatonetti_Lab/20231113_onsides/adverse_reactions_active_labels.csv')
meddra_pts = adr_section[['pt_meddra_term', 'pt_meddra_id']].drop_duplicates()
print(meddra_pts.shape)

(4370, 2)


In [28]:
def gpt_call(api_source, config, gpt_model, system_content, prompt, meddra_term, condition, temperature):
    if api_source == 'OpenAI':
        client = OpenAI(api_key=config[api_source]['openai_api_key'])
    elif api_source == 'Azure':
        client = AzureOpenAI(api_key=config[api_source]['openai_api_key'], api_version="2023-12-01-preview", azure_endpoint=config[api_source]['openai_api_endpoint'])
    else:
        raise Exception(f"Unexpected API source requested: {api_source}")
  
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_content},
            {
                "role": "user",
                "content": prompt.format(meddra_term, condition)
            }
        ],
        model=gpt_model,
        temperature=temperature,
    )
    term = chat_completion.choices[0].message.content
    return term

In [34]:
rows_to_run = meddra_pts.values.tolist()

In [29]:
def run_iteration(row):
    meddra_term = row[0]
    meddra_id = row[1]

    response = []
    for condition in ['Acute Kidney Injury', 'Acute Liver Injury',
                       'Acute Myocardial Infarction', 'GI bleeding']:    
        try:
            response.append(gpt_call(api_source, config, gpt_model, system_content,
                                      prompt, meddra_term, condition, temperature))
        except Exception as err:
            print(f"Encountered an exception for row: {meddra_term} {condition}. Error message below:")
            print(err)
            response.append([np.nan])
        
    return [meddra_term, meddra_id] + response

In [79]:
run_iteration(['Haematemesis', 1111111])
run_iteration(['acute liver injury', 1111111])

['acute liver injury', 1111111, 'False', 'True', 'False.', 'False']

In [36]:
output = {}
for i in range(nruns):    
    results = list()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as exec:
        results.extend(list(tqdm(
            exec.map(run_iteration, rows_to_run), 
            total=len(rows_to_run)
        )))

    output[i] = results

100%|██████████| 4370/4370 [53:52<00:00,  1.35it/s]  


In [39]:
output[1][0]

['Abdominal discomfort',
 10000059,
 'False.',
 'False. Abdominal discomfort is a symptom that can be associated with a wide range of conditions, not specifically an example of acute liver injury. Acute liver injury refers to sudden damage to the liver, which can have various causes and may present with symptoms such as jaundice, nausea, vomiting, and abdominal pain, among others.',
 'False. Abdominal discomfort is a symptom that can be associated with many conditions, but it is not an example of Acute Myocardial Infarction (AMI). AMI, also known as a heart attack, may sometimes present with abdominal discomfort, but the discomfort is not definitive of the condition itself. AMI is primarily characterized by the sudden blockage of blood flow to a part of the heart muscle, typically resulting in chest pain, shortness of breath, and other cardiac-related symptoms.',
 'False']

In [61]:
# ['Acute Kidney Injury', 'Acute Liver Injury',
#                        'Acute Myocardial Infarction', 'GI bleeding']
final = []
for key, r in output.items():
    final.extend([[key] + res for res in r])
gpt_output = pd.DataFrame(final, 
             columns=['run', 'meddra_term', 'meddra_pt_id', 'AKI',  'ALI', 'AMI', 'GIB'])

In [ ]:
gpt_output['AKI'] = [1 if 'true' in x.lower() else 0 for x in gpt_output['AKI']]
gpt_output['ALI'] = [1 if 'true' in x.lower() else 0 for x in gpt_output['ALI']]
gpt_output['AMI'] = [1 if 'true' in x.lower() else 0 for x in gpt_output['AMI']]
gpt_output['GIB'] = [1 if 'true' in x.lower() else 0 for x in gpt_output['GIB']]


In [75]:
gpt_output.to_csv('data/gpt_annotation_meddra_pt.csv', index=False)

In [66]:
# group by meddra term and sum the number of true statements
gpt_output.groupby(['meddra_term', 'meddra_pt_id']).sum().reset_index().head()

,meddra_term,meddra_pt_id,run,AKI,ALI,AMI,GIB
0,Abdominal abscess,10060921,3,0,0,0,0
1,Abdominal adhesions,10000050,3,0,0,0,0
2,Abdominal discomfort,10000059,3,0,0,0,0
3,Abdominal distension,10000060,3,0,0,0,0
4,Abdominal hernia,10060954,3,0,0,0,0


In [71]:
gpt_output.groupby(['meddra_term', 'meddra_pt_id']).sum().reset_index().query('GIB > 0')

,meddra_term,meddra_pt_id,run,AKI,ALI,AMI,GIB
177,Anal fissure,10002153,3,0,0,0,3
180,Anal haemorrhage,10049555,3,0,0,0,3
184,Anal ulcer,10002180,3,0,0,0,3
193,Anastomotic ulcer,10002243,3,0,0,0,3
892,Colitis ischaemic,10009895,3,0,0,0,3
894,Colitis ulcerative,10009900,3,0,0,0,3
1146,Diarrhoea haemorrhagic,10012741,3,0,0,0,3
1177,Diverticulum intestinal haemorrhagic,10013560,3,0,0,0,3
1204,Duodenal ulcer,10013836,3,0,0,0,3
1205,Duodenal ulcer haemorrhage,10013839,3,0,0,0,3
